# Technical Agent 技術分析測試

本測試展示：**技術分析 Agent 的完整工作流程**

使用 5 個自訂技術分析工具（@function_tool）進行手動測試：
- calculate_technical_indicators
- identify_chart_patterns
- analyze_trend
- analyze_support_resistance
- generate_trading_signals

## Step 1: 環境設置

In [1]:
import sys
from pathlib import Path
from dotenv import load_dotenv

from agents import (
    Runner,
    gen_trace_id,
    trace,
    WebSearchTool,
    CodeInterpreterTool,
)
from agents.mcp import MCPServerStdio

load_dotenv()

project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root / 'backend' / 'src'))

from common.logger import logger

print('✓ 環境設置完成')

2025-10-19 18:18:57 | INFO     | common.logger:setup_logger:100 | Logger initialized with level: INFO
2025-10-19 18:18:57 | INFO     | common.logger:setup_logger:102 | Logging to file: /Users/sacahan/Documents/workspace/CasualTrader/backend/logs/casualtrader.log


✓ 環境設置完成


## Step 2: 導入必要的模組

In [2]:
from trading.tools.technical_agent import (
    get_technical_agent,
)

print('✓ 模組導入完成')

✓ 模組導入完成


## Step 3: 初始化 OpenAI 內建工具

In [3]:
# WebSearchTool: 提供網路搜尋功能
web_search_tool = WebSearchTool(
    user_location=None,  # 可選：用戶位置，用於本地化搜尋結果
    filters=None,  # 可選：搜尋過濾器
    search_context_size="medium",  # 搜尋上下文大小：'low'、'medium'、'high'
)

# CodeInterpreterTool: 提供程式碼執行功能
# 必須指定 type 和 container 設置，container.type 必須為 "auto"
code_interpreter_tool = CodeInterpreterTool(
    tool_config={
        "type": "code_interpreter",
        "container": {
            "type": "auto"  # OpenAI 自動選擇最適合的容器
        },
    }
)

openai_tools = [web_search_tool, code_interpreter_tool]

## Step 4: 初始化 MCP Servers

In [4]:
# 全局變量來管理 MCP servers
casual_market_mcp = None
mcp_memory = None
mcp_servers = []

async def setup_mcp_servers():
    """設置 MCP servers"""
    global casual_market_mcp, mcp_memory, mcp_servers
    
    # 清理舊的連接
    if casual_market_mcp:
        try:
            await casual_market_mcp.__aexit__(None, None, None)
        except:
            pass
    
    if mcp_memory:
        try:
            await mcp_memory.__aexit__(None, None, None)
        except:
            pass
    
    # 初始化 casual-market-mcp
    casual_market_mcp = MCPServerStdio(
        params={
            "command": "uvx",
            "args": [
                "--from",
                "/Users/sacahan/Documents/workspace/CasualMarket",
                "casual-market-mcp",
            ],
        }, 
        name="casual_market_mcp", 
        client_session_timeout_seconds=120  # 增加超時時間
    )
    
    # 初始化 mcp-memory-libsql
    mcp_memory = MCPServerStdio(
        params={
            "command": "npx",
            "args": ["-y", "mcp-memory-libsql"],
            "env": {"LIBSQL_URL": "file:./labs/technical_agent.db"},
        },
        name="mcp_memory",
        client_session_timeout_seconds=120,  # 增加超時時間
    )
    
    # 連接 servers
    await casual_market_mcp.__aenter__()
    await mcp_memory.__aenter__()
    
    mcp_servers = [casual_market_mcp, mcp_memory]
    print('✓ MCP Servers 已連接')

await setup_mcp_servers()

✓ MCP Servers 已連接


## Step 5: 初始化 Technical Agent

In [ ]:
# 創建技術分析 Agent
technical_agent = await get_technical_agent(
    model_name="gpt-5-mini",
    mcp_servers=mcp_servers,
    openai_tools=openai_tools
)

print('✓ Technical Agent 已初始化')

2025-10-19 18:19:16 | INFO     | trading.tools.technical_agent:get_technical_agent:491 | get_technical_agent() called with model=gpt-5-mini
2025-10-19 18:19:16 | INFO     | trading.tools.technical_agent:get_technical_agent:506 | Creating Agent with model=gpt-5-mini, mcp_servers=2, tools=7
2025-10-19 18:19:16 | INFO     | trading.tools.technical_agent:get_technical_agent:519 | Technical Analyst Agent created successfully


✓ Technical Agent 已初始化


## 測試 1: 單一股票技術分析

In [ ]:
async def test_single_stock_analysis():
    print('\n' + '='*70)
    print('測試 1: 單一股票技術分析 - 台積電 (2330)')
    print('='*70)
    
    prompt = '''請進行台積電 (2330) 的技術分析。
    
最近 20 日價格數據（由近到遠）：
日期      開盤   高     低     收盤   成交量
2024-10-18  450   455   448   452   1,200
2024-10-17  448   453   445   450   1,100
2024-10-16  445   450   442   448   1,050
2024-10-15  440   445   438   444   980
2024-10-14  435   442   432   440   950
2024-10-11  430   438   428   435   920
2024-10-10  425   433   423   430   900
2024-10-09  420   428   418   425   880
2024-10-08  415   423   413   420   850
2024-10-07  410   418   408   415   820
2024-10-04  405   413   402   410   800
2024-10-03  400   408   398   405   780
2024-10-02  395   403   393   400   760
2024-10-01  390   398   388   395   740
2024-09-30  385   393   383   390   720
2024-09-27  380   388   378   385   700
2024-09-26  375   383   373   380   680
2024-09-25  370   378   368   375   660
2024-09-24  365   373   363   370   640
2024-09-23  360   368   358   365   620

請進行以下分析：
1. 計算關鍵技術指標（MA、RSI、MACD）
2. 識別圖表型態
3. 分析趨勢方向和強度
4. 找出支撐和壓力位
5. 產生交易訊號和建議'''
    
    print(f'\n用戶提示:\n{prompt}')
    print('\n→ Agent 執行分析...')
    print('-' * 70)
    
    trace_id = gen_trace_id()
    try:
        with trace(workflow_name="technical_agent_lab", trace_id=trace_id):
            result = await Runner.run(technical_agent, prompt, max_turns=15)
    
        print('-' * 70)
        print('\nAgent 分析結果:')
        print(result.final_output)
        
        return result
    except Exception as e:
        print(f"執行錯誤: {e}")
        print(f"錯誤類型: {type(e).__name__}")
        return None

result_1 = await test_single_stock_analysis()


測試 1: 單一股票技術分析 - 台積電 (2330)

用戶提示:
請進行台積電 (2330) 的技術分析。

最近 20 日價格數據（由近到遠）：
日期      開盤   高     低     收盤   成交量
2024-10-18  450   455   448   452   1,200
2024-10-17  448   453   445   450   1,100
2024-10-16  445   450   442   448   1,050
2024-10-15  440   445   438   444   980
2024-10-14  435   442   432   440   950
2024-10-11  430   438   428   435   920
2024-10-10  425   433   423   430   900
2024-10-09  420   428   418   425   880
2024-10-08  415   423   413   420   850
2024-10-07  410   418   408   415   820
2024-10-04  405   413   402   410   800
2024-10-03  400   408   398   405   780
2024-10-02  395   403   393   400   760
2024-10-01  390   398   388   395   740
2024-09-30  385   393   383   390   720
2024-09-27  380   388   378   385   700
2024-09-26  375   383   373   380   680
2024-09-25  370   378   368   375   660
2024-09-24  365   373   363   370   640
2024-09-23  360   368   358   365   620

請進行以下分析：
1. 計算關鍵技術指標（MA、RSI、MACD）
2. 識別圖表型態
3. 分析趨勢方向和強度
4. 找出支撐和壓力位
5. 產生交易訊號和建議


2025-10-19 18:19:33 | INFO     | httpx._client:_send_single_request:1025 | HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
2025-10-19 18:19:46 | INFO     | httpx._client:_send_single_request:1740 | HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-10-19 18:19:46 | INFO     | trading.tools.technical_agent:calculate_technical_indicators:185 | 開始計算技術指標 | 股票: 2330 | 數據點數: 20
2025-10-19 18:19:46 | INFO     | trading.tools.technical_agent:calculate_technical_indicators:232 | 技術指標計算完成 | 股票: 2330 | 指標數: 5
2025-10-19 18:19:49 | INFO     | httpx._client:_send_single_request:1025 | HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
2025-10-19 18:20:00 | INFO     | httpx._client:_send_single_request:1740 | HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-10-19 18:20:00 | INFO     | trading.tools.technical_agent:identify_chart_patterns:265 | 開始識別圖表型態 | 股票: 2330 | 數據點數: 20 

----------------------------------------------------------------------

Agent 分析結果:
下面是基於你提供的最近 20 日（最近日為 2024-10-18）台積電（2330）價格資料所做的技術分析。注意：本分析完全以你提供的資料（截至 2024-10-18，收盤價 452）為依據；若要做交易決策，建議先比對當前即時報價與當日成交量/消息。

一、關鍵技術指標（計算結果）
- 移動平均線（MA）
  - MA5  = 442.96
  - MA10 = 438.44
  - MA20 = 429.40
  - MA60 = 415.84
- RSI（14）: 55.0 — 中性（未過買或過賣區）
- MACD:
  - MACD 線 = 0.50
  - Signal 線 = 0.30
  - Histogram = 0.20 → 多頭（MACD > signal，histogram 正值表示動能在擴張）
- 布林通道（20）
  - 上軌 = 461.04
  - 中軌 = 452.00
  - 下軌 = 442.96
  - 備註：當前收盤 452 剛好接近中軌
- KD
  - K = 60, D = 55 → 偏強但未過熱

二、圖表型態識別
- 識別結果：上升趨勢（bullish），信心水準約 75%（工具判斷，說明最近一段為穩健上升趨勢，約 +23.8% 的漲幅區間）
- 觀察：價格在過去 20 日呈現連續向上推進，形成短中期上升趨勢，無明顯頭肩或三角等反轉型態。

三、趨勢方向與強度
- 趨勢方向：上升
- 強度評估：0.8（0-1 範圍，偏強）
- 動能：
  - 短期動能 ≈ +6.0%（短期正向）
  - 中期動能 ≈ +13.7%（中期上行動能更強）
- 判斷要點：價格位於 MA5/10/20 之上；MACD 為多頭且 histogram 正值；RSI 在中性區，不顯示過熱 → 支持趨勢延續的可能性。

四、支撐與壓力位（工具計算）
- 當前價格（依資料） = 452.00（2024-10-18 收盤）
- 支撐位（由近到遠）
  1. 429.40（接近 MA20）
  2. 415.84（MA60）
  3. 406.80（次級支撐）
- 壓力位（由近到遠）
  1

2025-10-19 18:21:23 | INFO     | httpx._client:_send_single_request:1025 | HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
2025-10-19 18:22:50 | INFO     | httpx._client:_send_single_request:1025 | HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
2025-10-19 18:23:11 | INFO     | httpx._client:_send_single_request:1025 | HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
2025-10-19 18:23:47 | INFO     | httpx._client:_send_single_request:1025 | HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"


## 測試 2: 完整 Agent 工作流程 - 短期交易機會

In [ ]:
async def test_short_term_trading():
    print('\n' + '='*70)
    print('測試 2: 完整 Agent 工作流程 - 短期交易機會分析')
    print('='*70)
    
    prompt = '''分析聯發科 (2454) 的短期交易機會。
    
當前技術面數據：
- 目前股價：900 元
- 10 日均線：885 元
- 20 日均線：870 元
- 60 日均線：850 元

技術指標：
- RSI(14)：65（偏強）
- MACD：正值且上升
- KD 快線：75，慢線：70（都在高檔）
- 布林通道：股價在上緣附近

最近走勢：
- 過去 5 日上漲 4.5%
- 成交量溫和增加
- 突破前高 895 元
- 技術型態：剛剛完成底部形成

市場環境：
- 大盤處於上升趨勢
- 科技股領漲
- 產業景氣向好

請分析：
1. 短期趨勢評估
2. 目前是否適合進場？
3. 建議的進場、停損、獲利了結點位
4. 風險/報酬比評估
5. 相關注意事項'''
    
    print(f'\n用戶提示:\n{prompt}')
    print('\n→ Agent 執行分析...')
    print('-' * 70)
    
    trace_id = gen_trace_id()
    try:
        with trace(workflow_name="technical_agent_lab", trace_id=trace_id):
            result = await Runner.run(technical_agent, prompt, max_turns=15)
    
        print('-' * 70)
        print('\nAgent 分析結果:')
        print(result.final_output)
        
        return result
    except Exception as e:
        print(f"執行錯誤: {e}")
        print(f"錯誤類型: {type(e).__name__}")
        return None

result_2 = await test_short_term_trading()

## 測試 3: 完整 Agent 工作流程 - 長期趨勢判斷

In [7]:
async def test_long_term_trend():
    print('\n' + '='*70)
    print('測試 3: 完整 Agent 工作流程 - 長期趨勢判斷')
    print('='*70)
    
    prompt = '''請進行廣達電腦 (2382) 的長期技術分析。
    
長期技術面數據（月線圖）：
- 過去 12 個月股價範圍：130-160 元
- 目前股價：155 元
- 52 週新高：160 元（最近達到）
- 52 週新低：130 元

長期均線情況：
- 200 日均線：145 元（上升趨勢）
- 100 日均線：150 元（上升趨勢）
- 50 日均線：152 元（上升趨勢）
- 股價處於所有均線之上

技術指標（月線）：
- MACD：金叉，距離越來越遠
- RSI：50-60 區間（中立偏強）
- 成交量：逐月增加

關鍵技術位：
- 強支撐位：140 元（前低）
- 重要阻力：165 元（前高）
- 突破點：160 元（新高剛剛達成）

請分析：
1. 長期趨勢是否改變？
2. 上升趨勢的強度和可持續性
3. 未來 6-12 個月的潛在目標位
4. 關鍵支撐和阻力位置
5. 長期投資者的買進機會和持有建議'''
    
    print(f'\n用戶提示:\n{prompt}')
    print('\n→ Agent 執行分析...')
    print('-' * 70)
    
    trace_id = gen_trace_id()
    try:
        with trace(workflow_name="technical_agent_lab", trace_id=trace_id):
            result = await Runner.run(technical_agent, prompt, max_turns=15)
    
        print('-' * 70)
        print('\nAgent 分析結果:')
        print(result.final_output)
        
        return result
    except Exception as e:
        print(f"執行錯誤: {e}")
        print(f"錯誤類型: {type(e).__name__}")
        return None
    
    return result

result_3 = await test_long_term_trend()


測試 3: 完整 Agent 工作流程 - 長期趨勢判斷

用戶提示:
請進行廣達電腦 (2382) 的長期技術分析。

長期技術面數據（月線圖）：
- 過去 12 個月股價範圍：130-160 元
- 目前股價：155 元
- 52 週新高：160 元（最近達到）
- 52 週新低：130 元

長期均線情況：
- 200 日均線：145 元（上升趨勢）
- 100 日均線：150 元（上升趨勢）
- 50 日均線：152 元（上升趨勢）
- 股價處於所有均線之上

技術指標（月線）：
- MACD：金叉，距離越來越遠
- RSI：50-60 區間（中立偏強）
- 成交量：逐月增加

關鍵技術位：
- 強支撐位：140 元（前低）
- 重要阻力：165 元（前高）
- 突破點：160 元（新高剛剛達成）

請分析：
1. 長期趨勢是否改變？
2. 上升趨勢的強度和可持續性
3. 未來 6-12 個月的潛在目標位
4. 關鍵支撐和阻力位置
5. 長期投資者的買進機會和持有建議

→ Agent 執行分析...
----------------------------------------------------------------------


2025-10-19 18:23:07 | INFO     | httpx._client:_send_single_request:1740 | HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-10-19 18:23:07 | INFO     | trading.tools.technical_agent:analyze_trend:327 | 開始分析趨勢 | 股票: 2382 | 數據點數: 24
2025-10-19 18:23:07 | INFO     | trading.tools.technical_agent:analyze_trend:345 | 趨勢分析完成 | 股票: 2382 | 方向: 上升 | 強度: 0.80
2025-10-19 18:23:44 | INFO     | httpx._client:_send_single_request:1740 | HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


----------------------------------------------------------------------

Agent 分析結果:
下面是針對您給出的廣達電腦（2382）「長期（月線）技術面」整理與結論。我先說明：我用您提供的月線要點作為主要依據，並用一個與您敘述相符的合成月線資料做趨勢量化檢測（分析工具輸出：方向 = 上升，強度 = 0.80，短期動能 ≈ 0.034， 中期動能 ≈ 0.080）。若您想要完全以真實歷史 OHLC 資料重新計算指標，我可以幫您抓取並做更精準回測／計算（需抓取實際資料）。

摘要重點（基於您提供的數據）
- 價格在 130–160 元區間過去 12 個月，現價 155 元，剛觸及 52 週高 160 元（最近達到）。
- 股價位於 50/100/200 日均線之上，三條均線均呈上升（多頭排列）。
- 月線 MACD 出現金叉且距離擴大；RSI 在 50–60（中立偏強）；成交量逐月增加。
整體結論：長期趨勢仍為上升趨勢，趨勢強度偏強且具延續條件，但仍須注意關鍵支撐位與量能變化。

逐題回答
1) 長期趨勢是否改變？
- 結論：尚未改變。理由：股價位於所有長期均線之上，均線走升、排列多頭，MACD 金叉且柱狀擴大，成交量逐月增加——這些是典型的長期多頭訊號。  
- 需注意的轉趨風險訊號：若月線收盤明顯跌破 200 日均線（約 145 元）並伴隨放量賣壓，或 MACD 由金叉轉回死叉並且量能萎縮，才會增加趨勢反轉的可能性。

2) 上升趨勢的強度和可持續性
- 強度量化（工具輸出）：strength ≈ 0.80（尺度 0-1，0.8 屬於強）。短中期動能皆為正，且中期動能高於短期，代表趨勢具有延續性而非短暫彈升。  
- 技術面支持可持續性的因素：均線向上、MACD 擴大、RSI 在中性偏強未顯著過熱、成交量逐月增加（量價配合）。  
- 主要不確定因素：若量能突然萎縮、或宏觀/產業基本面（伺服器/筆電/代工需求等）突變，才會削弱可持續性。

3) 未來 6–12 個月的潛在目標位（多種情境）
- 保守情境（維持現有上升動能但突破力度有限）：若拉回再反彈，短期先挑戰 160（已達）→ 165（重要前高阻力）→ 170。合理目標區：165–175。  
- 進取情境（突破 160 並放量確認）：若突

## 清理資源

In [ ]:
async def cleanup_mcp_servers():
    """清理 MCP servers 連接"""
    global casual_market_mcp, mcp_memory
    
    try:
        if casual_market_mcp:
            await casual_market_mcp.__aexit__(None, None, None)
            print('✓ casual_market_mcp 已關閉')
        
        if mcp_memory:
            await mcp_memory.__aexit__(None, None, None)
            print('✓ mcp_memory 已關閉')
            
        print('✓ 所有 MCP servers 已清理完成')
        
    except Exception as e:
        print(f'清理過程中發生錯誤: {e}')

# 測試完成後執行清理
await cleanup_mcp_servers()